In [ ]:
# ========================================================
# Importação de bibliotecas
# ========================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import inspect
import os
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# ========================================================

In [ ]:
# ========================================================
# Carregamento do dataset
dataset = pd.read_csv("../data/creditcard.csv")

# Converte todos os nomes de colunas para minúsculas
dataset.columns = dataset.columns.str.lower()

# Criação do diretório para salvar gráficos
os.makedirs('graficos', exist_ok=True)

# ========================================================

In [ ]:
# ============================================================
#                PASSO 1 — Análise Exploratória de Dados (EDA)
# ============================================================

# Nesta etapa, realizamos uma avaliação inicial do conjunto de dados com o objetivo de compreender melhor suas características e identificar
# padrões importantes para o problema de detecção de fraudes. Foram analisadas as distribuições das variáveis, a presença de valores 
# extremos, o comportamento das transações ao longo do tempo e as diferenças entre operações

# ========================================================

In [ ]:
# ========================================================
# Visualizar as primeiras 5 linhas do dataset
dataset.head()

# ========================================================

In [ ]:
# ========================================================
# Visualizar as Colunas do dataset
dataset.info()

# ========================================================

In [ ]:
# ========================================================
# Estatísticas descritivas
# Mostra medidas como média, desvio padrão, mínimos, máximos e quartis das variáveis numéricas.
# Útil para entender a distribuição dos dados, detectar outliers e avaliar necessidade de normalização.

dataset.describe()

# ========================================================

In [ ]:
# ========================================================
# Contagem de classes
# Mostra quantas transações pertencem a cada classe (0 = não fraude, 1 = fraude).
# Útil para verificar o desbalanceamento do dataset e entender a proporção de fraudes.

print(dataset['class'].value_counts())

# ========================================================

In [ ]:
# ========================================================
# Distribuição das classes
# Cria um gráfico de barras mostrando a quantidade de transações por classe
# (0 = não fraude, 1 = fraude).
# Útil para visualizar o desbalanceamento do dataset e confirmar a baixa proporção de fraudes.
# Os números exatos de casos são exibidos no topo de cada barra.

plt.figure(figsize=(6,5))
ax = sns.countplot(x='class', data=dataset, palette="viridis")
plt.title("Distribuição das classes (0 = não fraude, 1 = fraude)")
plt.xlabel("Classe")
plt.ylabel("Quantidade de transações")
plt.savefig('graficos/Distribuicao_classes.png', dpi=300, bbox_inches='tight')

# Adicionar valores exatos em cima das barras
for p in ax.patches:
    ax.annotate(str(p.get_height()), 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='bottom', fontsize=10, fontweight='bold', rotation=0)

plt.tight_layout()
plt.show()

# ========================================================

In [ ]:
# ========================================================
# Boxplot do valor das transações
# Mostra a distribuição dos valores das transações, destacando concentração em valores baixos e presença de outliers.
# Além do boxplot, adiciona os principais valores estatísticos (mínimo, quartis e máximo) diretamente no gráfico.

plt.figure(figsize=(12,6))
ax = sns.boxplot(x=dataset['amount'], color="skyblue")
plt.title("Boxplot do valor das transações")
plt.savefig('graficos/Boxplot do valor das transações.png', dpi=300, bbox_inches='tight')

# Estatísticas
stats = dataset['amount'].describe()

# Posições Y separadas (evita bagunça)
y_positions = {
    'min': 0.15,
    '25%': 0.35,
    '50%': 0.45,
    '75%': 0.55,
    'max': 0.65
}

# Adicionar anotações sem empilhar tudo
for stat_name, stat_value in stats[['min','25%','50%','75%','max']].items():
    ax.annotate(
        f"{stat_name}: {stat_value:.2f}",
        xy=(stat_value, 0),
        xytext=(stat_value, y_positions[stat_name]),
        textcoords='data',
        ha='center',
        fontsize=9,
        fontweight='bold',
        rotation=45,
        arrowprops=dict(arrowstyle="->", color="black")
    )

plt.show()

# ========================================================

In [ ]:
# ========================================================
# Distribuição do tempo das transações
# Cria um histograma para visualizar a frequência de transações ao longo do tempo (em segundos).
# Útil para identificar picos de atividade, padrões temporais e possíveis concentrações de transações.
# O gráfico foi ajustado com estilo consistente e contagem exata no topo de cada barra.

plt.figure(figsize=(12,6))
ax = sns.histplot(dataset['time'], bins=100, kde=False, color="skyblue")
plt.title("Distribuição do tempo das transações")
plt.xlabel("Segundos desde o início da coleta")
plt.ylabel("Quantidade de transações")
plt.savefig('graficos/Distribuição do tempo das transações.png', dpi=300, bbox_inches='tight')

# Adicionar valores exatos no topo de cada barra
for patch in ax.patches:
    height = patch.get_height()
    if height > 0:
        ax.annotate(f'{int(height)}',
                    (patch.get_x() + patch.get_width() / 2, height),
                    ha='center', va='bottom',
                    fontsize=8, fontweight='bold', rotation=90)

plt.tight_layout()
plt.show()

# ========================================================

In [ ]:
# ========================================================
# Matriz de correlação entre variáveis (somente numéricas)
# Remove colunas não numéricas (como 'periodo') e calcula a correlação apenas entre variáveis numéricas.
# Útil para identificar relações lineares entre atributos quantitativos e a variável alvo (class).

plt.figure(figsize=(14,10))

corr = dataset.select_dtypes(include=['number']).corr()

sns.heatmap(
    corr,
    cmap="coolwarm",
    center=0,
    annot=True,
    fmt=".2f",
    annot_kws={"size": 6},   # texto menor = heatmap mais legível
    linewidths=0.5,
    cbar_kws={"shrink": 0.8}
)

plt.title("Matriz de correlação entre variáveis (numéricas)", fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right', fontsize=6)
plt.yticks(rotation=0, fontsize=6)
plt.tight_layout()
plt.show()

# ========================================================

In [ ]:
# ========================================================
# Criar coluna (hora) a partir da variável (time)
dataset['hora'] = dataset['time'] / 3600

def periodo_do_dia(hora):
    hora = int(hora % 24)  # garante que fique dentro de 0–23h
    if 6 <= hora < 12:
        return 'manhã'
    elif 12 <= hora < 18:
        return 'tarde'
    elif 18 <= hora < 24:
        return 'noite'
    else:
        return 'madrugada'

# Criar coluna categórica (periodo)
dataset['periodo'] = dataset['hora'].apply(periodo_do_dia)

# One-hot encoding limpo
dataset_encoded = pd.get_dummies(dataset, columns=['periodo'], prefix='periodo')

# ======= GRÁFICO DE CORRELAÇÃO AJUSTADO =======

plt.figure(figsize=(14, 10))

corr = dataset_encoded.corr()

sns.heatmap(
    corr,
    cmap="coolwarm",
    center=0,
    annot=True,
    fmt=".2f",
    annot_kws={"size": 8},       # deixa legível
    linewidths=0.5,
    cbar_kws={"shrink": 0.7}     # barra menor
)

plt.title(
    "Matriz de Correlação entre Variáveis (com período codificado)",
    fontsize=16,
    fontweight='bold'
)

plt.xticks(rotation=45, ha='right', fontsize=9)
plt.yticks(rotation=0, fontsize=9)

plt.tight_layout()

# Salvar na pasta já criada
plt.savefig(
    'graficos/matriz_correlacao_periodo.png',
    dpi=300,
    bbox_inches='tight'
)

plt.show()

# ========================================================

In [ ]:
# ========================================================
# Distribuição das transações por hora
# Converte o tempo de segundos para horas e agrupa por hora inteira.
# Mostra em gráfico de barras o volume de transações ao longo das 48 horas de coleta.
# Útil para identificar picos de atividade e padrões temporais.
# Os números exatos de transações são exibidos no topo de cada barra.
# Converter tempo para horas
dataset['hora'] = dataset['time'] / 3600

# Agrupar por hora inteira
dataset['hora_inteira'] = dataset['hora'].astype(int)
horas = dataset['hora_inteira'].value_counts().sort_index()

# Plotar gráfico de barras
plt.figure(figsize=(14,6))
sns.barplot(x=horas.index, y=horas.values, palette="viridis")
plt.title("Distribuição das transações por hora")
plt.savefig('graficos/Distribuição das transações por hora.png', dpi=300, bbox_inches='tight')
plt.xlabel("Hora desde o início da coleta")
plt.ylabel("Quantidade de transações")

# Adicionar número exato de transações no topo de cada barra com rotação
for i, val in enumerate(horas.values):
    plt.text(i, val + 100, str(val), ha='center', va='bottom', fontsize=9, fontweight='bold', rotation=45)

plt.xticks(rotation=0, fontsize=9)  # mantém as horas na horizontal
plt.tight_layout()
plt.show()

# ========================================================

In [ ]:
# ========================================================
# Distribuição das transações por período do dia
# Converte o tempo em horas e classifica em ciclos de 24h (madrugada, manhã, tarde, noite).
# Cria uma nova coluna 'periodo' e conta a quantidade de transações em cada faixa.
# Útil para identificar padrões de atividade em diferentes períodos do dia.
# Os valores exatos são exibidos no topo de cada barra.

# Converter segundos em horas
dataset['hora'] = dataset['time'] / 3600

# Função para categorizar períodos do dia
def periodo_do_dia(hora):
    hora = int(hora % 24)  # ciclo de 24h
    if 6 <= hora < 12:
        return 'manhã'
    elif 12 <= hora < 18:
        return 'tarde'
    elif 18 <= hora < 24:
        return 'noite'
    else:
        return 'madrugada'

# Criar coluna de período
dataset['periodo'] = dataset['hora'].apply(periodo_do_dia)

# Contar transações por período
periodos = dataset['periodo'].value_counts().reindex(['madrugada','manhã','tarde','noite'])

# Plotar gráfico
plt.figure(figsize=(8,6))
sns.barplot(x=periodos.index, y=periodos.values, palette="viridis")
plt.title("Distribuição das transações por período do dia")
plt.savefig('graficos/Distribuição das transações por período do dia.png', dpi=300, bbox_inches='tight')
plt.xlabel("Período do dia")
plt.ylabel("Quantidade de transações")

# Adicionar valores exatos em cima das barras
for i, val in enumerate(periodos.values):
    plt.text(i, val + 500, str(val), ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

# ========================================================

In [ ]:
# ============================================================
#                PASSO 2 — PRÉ-PROCESSAMENTO
# ============================================================

# O pré-processamento prepara o dataset para que o modelo aprenda de forma correta, eliminando 
# problemas como escalas diferentes, dados extremos e desbalanceamento. No nosso caso, serão aplicados os seguintes processos:
# ============================================================

# 2.1 — Verificação de valores ausentes
# 2.2 — Tratamento de valores ausentes
# 2.3 — Normalização (padronização) das colunas "amount" e "time"
# 2.4 — Remoção de outliers
# 2.5 — Balanceamento das classes
# 2.6 — Separação final em treino e testes
# ============================================================

In [ ]:
# ========================================================
#2.1 - Verificar valores ausentes
print("Valores ausentes por coluna:")
print(dataset.isnull().sum())

# ========================================================

In [ ]:
# ========================================================
#2.2 - Remove linhas com valores ausentes (caso existam)
dataset = dataset.dropna()
print("\nApós remoção de valores ausentes:", dataset.shape)

# ========================================================

In [ ]:
# ========================================================
# 2.3 — Normalização das colunas Amount e Time

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#Esse objeto calcula:
#média da coluna
#desvio padrão
#e transforma os valores usando a função z = (x - média) / desvio padrão

dataset[['amount_scaled', 'time_scaled']] = scaler.fit_transform(dataset[['amount', 'time']])
#média = 0
#desvio padrão = 1
#deixam o dataset mais adequado para treinar

print("\nColunas normalizadas adicionadas (amount_scaled, time_scaled).")
#Agora o dataset possui:
#amount_scaled
#time_scaled
#otal de 33 colunas agora.

# ========================================================

In [ ]:
# ========================================================
# 2.4 — Remover outliers da coluna 'amount'
# O dataset tem transações com valores muito altos, removemos extremos
# usando a técnica IQR (Intervalo Inter-Quartil).
Q1 = dataset['amount'].quantile(0.25)
Q3 = dataset['amount'].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

dataset = dataset[
    (dataset['amount'] >= limite_inferior) &
    (dataset['amount'] <= limite_superior)
]

print("\nApós remoção de outliers:", dataset.shape)

# ========================================================

In [ ]:
# ========================================================
# 2.5 - Balanceamento das classes com SMOTE
# O dataset é extremamente desbalanceado (fraudes ≈ 0.17%)

from imblearn.over_sampling import SMOTE

# Separar atributos (X) e rótulo (y)
# Removemos 'class' (alvo) e 'periodo' (categórica) para evitar erro no SMOTE
X = dataset.drop(['class', 'periodo'], axis=1)
y = dataset['class']

# O SMOTE cria novos exemplos sintéticos da classe minoritária (fraudes),
# usando interpolação entre vizinhos próximos. Isso evita overfitting e
# deixa as duas classes com o mesmo número de amostras.
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Mostrar distribuição antes e depois do balanceamento
print("\nDistribuição antes do SMOTE:")
print(y.value_counts())

print("\nDistribuição depois do SMOTE:")
print(y_resampled.value_counts())

# ========================================================

In [ ]:
# ========================================================
# 2.6 — Separar os dados em treino e teste
# Usamos 80% para treino e 20% para teste

# Até aqui aplicamos SMOTE, então o dataset ficou balanceado:
# Classe 0 - 234.554
# Classe 1 - 234.554
# Total após SMOTE = 469.108 registros (aproximadamente)
#
# Depois dividimos os dados em:
# 80% para TREINO - modelo aprende
# 20% para TESTE  - modelo é avaliado
#
# OBS: os valores finais podem variar um pouco porque o SMOTE
# cria amostras sintéticas usando vizinhos próximos.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled,
    test_size=0.2,
    random_state=42
)

print("\nTamanho do conjunto de treino:", X_train.shape)
print("Tamanho do conjunto de teste:", X_test.shape)

# ========================================================

In [ ]:
# ========================================================
# Boxplot dos valores por classe
# Verifica se os outliers da coluna (amount) estão associados à classe de fraude.
# A análise mostra que os valores extremos estão majoritariamente na classe 0 (não fraude),
# indicando que os outliers não representam transações fraudulentas.
# Portanto, a remoção desses outliers foi considerada adequada para reduzir ruído sem
# comprometer a detecção de fraudes.

plt.figure(figsize=(12,6))
sns.boxplot(x='class', y='amount', data=dataset)
plt.title("Distribuição dos valores por classe (0 = não fraude, 1 = fraude)")
plt.savefig('graficos/Distribuição dos valores por classe (0 = não fraude, 1 = fraude).png', dpi=300, bbox_inches='tight')
plt.show()

# ========================================================

In [ ]:
# ============================================
# 3. MODELAGEM — Treinamento de três algoritmos
# ============================================

# MODELO 1: KNN (K-Nearest Neighbors)
# MODELO 2: ÁRVORE DE DECISÃO (Decision Tree)
# MODELO 3: Random Forest

# ========================================================

In [ ]:
# ========================================================
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold, cross_val_score
import matplotlib.pyplot as plt
import pandas as pd

# ========================================================
# MODELO 1: KNN
# ========================================================

# ========================================================
# O KNN classifica uma transação verificando os vizinhos
# mais próximos no espaço de atributos. É simples e serve
# como baseline para comparar com outros modelos.
# ========================================================


knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
y_proba_knn = knn.predict_proba(X_test)[:,1]

print("\n===== RESULTADOS DO MODELO KNN =====")
print("Acurácia:", accuracy_score(y_test, y_pred_knn))
print("ROC AUC:", roc_auc_score(y_test, y_proba_knn))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred_knn))
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred_knn))
# ========================================================

In [ ]:
# ========================================================
# MODELO 2: ÁRVORE DE DECISÃO (Decision Tree)
# ========================================================
# A árvore de decisão cria regras de decisão para classificar
# cada transação. É rápida, interpretável e eficiente.


tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)
y_proba_tree = tree.predict_proba(X_test)[:,1]

print("\n===== RESULTADOS DO MODELO DECISION TREE =====")
print("Acurácia:", accuracy_score(y_test, y_pred_tree))
print("ROC AUC:", roc_auc_score(y_test, y_proba_tree))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred_tree))
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred_tree))

# ========================================================

In [ ]:
# ========================================================
# MODELO 3: Random Forest
# ========================================================
# Árvores de decisão treinadas com amostras/atributos distintos.
# Geralmente reduz overfitting da árvore única e é muito eficaz em dados tabulares.


rf = RandomForestClassifier(
    n_estimators=100,       # reduz número de árvores
    min_samples_leaf=10,    # folhas maiores, menos complexidade
    n_jobs=-1,
    random_state=42
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:,1]

print("\n===== RESULTADOS DO MODELO RANDOM FOREST =====")
print("Acurácia:", accuracy_score(y_test, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test, y_proba_rf))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred_rf))
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred_rf))

# ========================================================

In [ ]:
# =============================================================
# 5. Comparação 
# =============================================================

# Nesta etapa, comparamos os três modelos aplicados: KNN, Decision Tree e Random Forest, avaliando desempenho
# limitações, robustez e aplicabilidade no contexto de detecção de fraudes. A análise considera não apenas métricas quantitativas
# mas também aspectos conceituais importantes para uso real em produção.

# Os três modelos apresentaram bons resultados, porém o Random Forest demonstrou
# superioridade em praticamente todas as métricas, além de ser mais estável e robusto. 
# O modelo é altamente eficaz para o cenário proposto de detecção de fraudes, embora melhorias adicionais possam tornar a solução ainda mais precisa e generalizável.
# =============================================================

In [ ]:
# ========================================================
# Importância de features
# ========================================================
importances = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print("\nTop 10 features por importância:\n", importances.head(10))

# ========================================================
# Validação cruzada com 3 folds (Random Forest otimizado)
# ========================================================
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
scores = cross_val_score(rf, X_resampled, y_resampled, cv=cv, scoring='f1')
print("\nF1 médio (Random Forest, CV=3):", scores.mean())

# ========================================================

In [ ]:
# ========================================================
# Curva ROC comparativa
# ========================================================
plt.figure(figsize=(10,6))

fpr_knn, tpr_knn, _ = roc_curve(y_test, y_proba_knn)
fpr_tree, tpr_tree, _ = roc_curve(y_test, y_proba_tree)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_proba_rf)

plt.plot(fpr_knn, tpr_knn, label=f"KNN (AUC = {roc_auc_score(y_test, y_proba_knn):.3f})")
plt.plot(fpr_tree, tpr_tree, label=f"Decision Tree (AUC = {roc_auc_score(y_test, y_proba_tree):.3f})")
plt.plot(fpr_rf, tpr_rf, label=f"Random Forest (AUC = {roc_auc_score(y_test, y_proba_rf):.3f})")

plt.plot([0,1],[0,1],'k--')  # linha diagonal
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Curva ROC Comparativa dos Modelos")
plt.savefig('graficos/Curva ROC Comparativa dos Modelos.png', dpi=300, bbox_inches='tight')
plt.legend()
plt.show()

# ========================================================

In [ ]:
# ========================================================
# Comparação final em tabela
# ========================================================

# - MODELO 3: Random Forest é o melhor modelo entre os três, com quase 100% de acurácia, recall e precisão.
# - Modelo extremamente robusto e confiável

# - MODELO 2: ÁRVORE DE DECISÃO - também teve desempenho excelente, porém, pode estar um pouco especialista demais (overfitting leve).

# - MODELO 1: KNN (K-Nearest Neighbors) funciona, mas é inferior e menos escalável, funciona comparando cada amostra com seus vizinhos mais próximos.


comparacao = pd.DataFrame({
    "Modelo": ["KNN", "Decision Tree", "Random Forest"],
    "Acurácia": [
        accuracy_score(y_test, y_pred_knn),
        accuracy_score(y_test, y_pred_tree),
        accuracy_score(y_test, y_pred_rf)
    ],
    "ROC AUC": [
        roc_auc_score(y_test, y_proba_knn),
        roc_auc_score(y_test, y_proba_tree),
        roc_auc_score(y_test, y_proba_rf)
    ],
    "Resumo": [
        "Bom, mas mais fraco e lento em grandes bases",
        "Excelente, mas pode sobreajustar",
        "Melhor modelo, quase nenhuma fraude perdida"
    ]
})

print("\nComparação final:\n", comparacao)

# ========================================================

In [ ]:
# ========================================================
# As matrizes de confusão permitem avaliar visualmente os acertos e erros de cada modelo.
# Elas mostram quantas transações legítimas (classe 0) e fraudulentas (classe 1) foram
# corretamente classificadas (diagonal principal) e quantas foram confundidas (fora da diagonal).
# Esse tipo de gráfico complementa as métricas numéricas (acurácia, precisão, recall, F1, ROC AUC),
# oferecendo uma visão intuitiva da capacidade dos modelos em distinguir fraude vs não fraude.

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Função para plotar matriz de confusão
def plot_confusion_matrix(y_true, y_pred, titulo):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.xlabel("Predito")
    plt.ylabel("Real")
    plt.title(titulo)
    plt.savefig('graficos/Matriz de Confusão - KNN.png', dpi=300, bbox_inches='tight')
    plt.savefig('graficos/Matriz de Confusão - Decision Tree.png', dpi=300, bbox_inches='tight')
    plt.savefig('graficos/Matriz de Confusão - Random Forest.png', dpi=300, bbox_inches='tight')
    plt.show()

# Matriz de confusão para cada modelo
plot_confusion_matrix(y_test, y_pred_knn, "Matriz de Confusão - KNN")
plot_confusion_matrix(y_test, y_pred_tree, "Matriz de Confusão - Decision Tree")
plot_confusion_matrix(y_test, y_pred_rf, "Matriz de Confusão - Random Forest")

# ========================================================

In [ ]:
# ========================================================
# Como melhoria implementamos um algoritmo mais avançado: XGBoost, resultando nos dados abaixo:

# ========================================================
# MODELO 4: XGBoost
# ========================================================
# O XGBoost é um algoritmo de boosting baseado em árvores de decisão.
# Ele constrói várias árvores sequenciais, onde cada nova árvore corrige
# os erros da anterior. É altamente eficiente e costuma ter desempenho
# superior em problemas de classificação tabular.

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 1) Instanciar o modelo com hiperparâmetros básicos
xgb = XGBClassifier(
    n_estimators=200,        # número de árvores
    learning_rate=0.1,       # taxa de aprendizado
    max_depth=6,             # profundidade máxima das árvores
    subsample=0.8,           # fração de amostras usadas em cada árvore
    colsample_bytree=0.8,    # fração de atributos usados em cada árvore
    random_state=42,
    n_jobs=-1,
    use_label_encoder=False,
    eval_metric="logloss"    # evita warnings
)

# 2) Treinar o modelo
xgb.fit(X_train, y_train)

# 3) Fazer previsões
y_pred_xgb = xgb.predict(X_test)
y_proba_xgb = xgb.predict_proba(X_test)[:,1]

# 4) Avaliar desempenho com métricas
print("\n===== RESULTADOS DO MODELO XGBOOST =====")
print("Acurácia:", accuracy_score(y_test, y_pred_xgb))
print("ROC AUC:", roc_auc_score(y_test, y_proba_xgb))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred_xgb))
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred_xgb))

# 5) Plotar matriz de confusão
cm_xgb = confusion_matrix(y_test, y_pred_xgb)
plt.figure(figsize=(6,4))
sns.heatmap(cm_xgb, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão - XGBoost")
plt.savefig('graficos/Matriz de Confusão - XGBoost.png', dpi=300, bbox_inches='tight')
plt.show()

# 6) Curva ROC exclusiva do XGBoost
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_proba_xgb)
plt.figure(figsize=(8,6))
plt.plot(fpr_xgb, tpr_xgb, label=f"XGBoost (AUC = {roc_auc_score(y_test, y_proba_xgb):.3f})")
plt.plot([0,1],[0,1],'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Curva ROC - XGBoost")
plt.savefig('graficos/Curva ROC - XGBoost.png', dpi=300, bbox_inches='tight')
plt.legend()
plt.show()

# 7) Importância das features
importances_xgb = pd.Series(xgb.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print("\nTop 10 features por importância (XGBoost):\n", importances_xgb.head(10))

# ========================================================

In [ ]:
# ========================================================
# Comparação de métricas entre os quatro modelos implementados: KNN, Decision Tree, Random Forest e XGBoost. 
import pandas as pd

# Criar DataFrame com métricas comparativas
comparacao = pd.DataFrame({
    "Modelo": ["KNN", "Decision Tree", "Random Forest", "XGBoost"],
    "Acurácia": [
        accuracy_score(y_test, y_pred_knn),
        accuracy_score(y_test, y_pred_tree),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_xgb)
    ],
    "ROC AUC": [
        roc_auc_score(y_test, y_proba_knn),
        roc_auc_score(y_test, y_proba_tree),
        roc_auc_score(y_test, y_proba_rf),
        roc_auc_score(y_test, y_proba_xgb)
    ],
})

print("\nTabela comparativa de métricas:\n")
print(comparacao)

# ========================================================